<a href="https://colab.research.google.com/github/nakamura196/ndl_ocr/blob/main/gcv2hocr%E3%81%AE%E5%AE%9F%E8%A1%8C%E3%82%B5%E3%83%B3%E3%83%97%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colabを用いたgcv2hocrの実行例

Google Cloud Vision APIの実行結果から透明テキスト付きPDFファイルを作成します。

参考：

https://github.com/dinosauria123/gcv2hocr.git

## APIキー

APIキーを入力してください。

In [ ]:
#@title

api_key = "<取得したAPIキー>" #@param {type:"string"}

In [ ]:
#@title 初期セットアップ
# google driveのマウント
from google.colab import drive
drive.mount('/content/drive/')

%cd /content
from IPython.display import clear_output 

# ライブラリのインストール
!pip install -q python-bidi
!pip install -q reportlab

# ソースコードのダウンロード
!git clone https://github.com/dinosauria123/gcv2hocr.git
%cd  gcv2hocr

# hocrファイルをpdfファイルに変換するスクリプトのダウンロード
# ダウンロード元のGitHubリポジトリ
# https://github.com/ocropus/hocr-tools
!wget https://raw.githubusercontent.com/ocropus/hocr-tools/master/hocr-pdf

# pdf
!apt-get install poppler-utils
!pip install -q pdf2image

# 変換
!apt install imagemagick

clear_output()

In [ ]:
#@title 独自関数のロード

from pathlib import Path
import glob
from tqdm import tqdm
from google.colab import files
import requests
import os
import time
from urllib import request
from pdf2image import convert_from_path
import datetime
import pytz

extMap = {
    "jpg": ["jpg", "jpeg", "JPG", "JPEG"],
    "png": ["png", "PNG"],
    "tif": ["tif", "TIF", "tiff", "TIFF"],
}

def createOutputDir(output_dir):
  _output_dir = output_dir
  if os.path.exists(output_dir):
    run_id = datetime.datetime.now(pytz.timezone('Asia/Tokyo')).strftime('%Y%m%d%H%M%S')
    _output_dir += "_" + run_id

  _output_dir = _output_dir + "/data"
  os.makedirs(_output_dir, exist_ok=True)
  return _output_dir

def exec(input_dir):
  target_files = []
  ext = ""
  for key in extMap:
    _target_files = glob.glob(input_dir + "/*." + key)
    if len(_target_files) > 0:
      target_files = _target_files
      ext = key
  for target_file in tqdm(target_files):
    output_path = target_file + ".json"
    if not os.path.exists(output_path):
      !./gcvocr.sh $target_file $api_key
    hocr_path = target_file.replace("." + ext, ".hocr")
    !python gcv2hocr.py $output_path > $hocr_path

  file_id = input_dir.split("/")[-2]
  pdf_path = "{}.pdf".format(input_dir.replace("/data", "/" + file_id))

  print()
  print("以下に出力しました。")
  print(pdf_path)
  !python hocr-pdf --savefile $pdf_path $input_dir
  files.download(pdf_path)

# 画像のダウンロード
def downloadImages(url, output_dir, process_size, time_sleep):
  print("### IIIFマニフェストを用いた画像のダウンロード ###")
  df = requests.get(url).json()
  canvases = df["sequences"][0]["canvases"]
  if process_size > 0:
    canvases = canvases[0:process_size]
  for i in tqdm(range(len(canvases))):
    res = canvases[i]["images"][0]["resource"]
    index = str(i+1).zfill(4)
    path = "{}/{}.jpg".format(output_dir, index)
    os.makedirs(os.path.dirname(path), exist_ok=True)
    url = canvases[i]["images"][0]["resource"]["@id"]

    time.sleep(time_sleep)
    request.urlretrieve(url, path)

def createImagesFromPDF(pdf_path, img_dir):
  print("### PDFファイルから画像ファイルの作成 ###")
  pdf_path = Path(pdf_path)
  img_path=Path(img_dir)
  convert_from_path(pdf_path, output_folder=img_path,fmt='jpeg', output_file=pdf_path.stem,dpi=100)

def fixExtension(path):
  root_ext_pair = os.path.splitext(path)
  ext_without_dot = root_ext_pair[1][1:]
  ext = ""
  for key in extMap:
    if ext_without_dot in extMap[key]:
      ext = key
  return root_ext_pair[0] + "." + ext

def convert2jpg(path):
  ext_without_dot = os.path.splitext(path)[1][1:]
  if ext_without_dot in ["png", "tif"]:
    output_path = path.replace("." + ext_without_dot, ".jpg")
    !convert $path $output_path
    return output_path
  return path

## 実行オプション

- 画像
  - [画像のURL](#scrollTo=XRx8WzzuL_U_)
  - [画像のアップロード](#scrollTo=rsjwYDeno51u)
- PDF
  - [PDFのURL](#scrollTo=CG9f0ISz0lC9)
  - [PDFのアップロード](#scrollTo=v4tcod72p-Jf)
- IIIF
  - [IIIF](#scrollTo=b90ky_C-MsOi)

## 画像のURL

入力サンプル：「校異源氏物語」（国立国会図書館所蔵）

In [ ]:
#@title 設定

url = "https://www.dl.ndl.go.jp/api/iiif/3437686/R0000006/full/full/0/default.jpg" #@param {type:"string"}
output_dir = "/content/drive/MyDrive/gcv2ocr/3437686" #@param {type:"string"}
output_dir = str(Path(output_dir))

In [ ]:
#@title 実行
_output_dir = createOutputDir(output_dir)

# 画像のダウンロード
output_path = _output_dir + "/" + url.split("/")[-1]
output_path = fixExtension(output_path)
output_path = convert2jpg(output_path)
!wget $url -O $output_path

exec(_output_dir)

## 画像のアップロード

In [ ]:
#@title 設定

output_dir = "/content/drive/MyDrive/gcv2ocr/img_upload" #@param {type:"string"}
output_dir = str(Path(output_dir))

In [ ]:
#@title 実行
_output_dir = createOutputDir(output_dir)

# 画像のアップロード
from google.colab import files
uploaded = files.upload()
input_file = next(iter(uploaded))

# 画像の移動
output_path = _output_dir + "/" + input_file
output_path = fixExtension(output_path)
output_path = convert2jpg(output_path)
!cp "$input_file" "$output_path"
!rm "$input_file"

exec(_output_dir)

## PDFのURL

入力サンプル：「東洋学芸雑誌」（人間文化研究機構国立国語研究所所蔵）

In [ ]:
#@title 設定

url = "https://dglb01.ninjal.ac.jp/ninjaldl/toyogakuge/001/PDF/tygz-001.pdf" #@param {type:"string"}
output_dir = "/content/drive/MyDrive/gcv2ocr/tygz-001" #@param {type:"string"}

output_dir = str(Path(output_dir))

In [ ]:
#@title 実行
_output_dir = createOutputDir(output_dir)

# ファイルのダウンロード
tmp_pdf_path = output_dir + "/raw.pdf"
!curl $url -o $tmp_pdf_path

# 画像の作成
createImagesFromPDF(tmp_pdf_path, _output_dir)

exec(_output_dir)

## PDFのアップロード

In [ ]:
#@title 設定

output_dir = "/content/drive/MyDrive/gcv2ocr/pdf_upload" #@param {type:"string"}

output_dir = str(Path(output_dir))

In [ ]:
#@title 実行
_output_dir = createOutputDir(output_dir)

# ファイルのアップロード
from google.colab import files
uploaded = files.upload()
input_file = next(iter(uploaded))

# ファイルの移動
tmp_pdf_path = output_dir + "/" + input_file
!cp "$input_file" "$tmp_pdf_path"
!rm "$input_file"

# 画像の作成
createImagesFromPDF(tmp_pdf_path, _output_dir)

exec(_output_dir)

## IIIF

入力サンプル：「校異源氏物語」（国立国会図書館所蔵）

In [ ]:
#@title 設定

url = "https://www.dl.ndl.go.jp/api/iiif/3437686/manifest.json" #@param {type:"string"}
output_dir = "/content/drive/MyDrive/gcv2ocr/3437686_iiif" #@param {type:"string"}

process_size =   5#@param {type:"number"}
sleep_time =   1#@param {type:"number"}

output_dir = str(Path(output_dir))

In [ ]:
#@title 実行
_output_dir = output_dir + "/data"

downloadImages(url, _output_dir, process_size, sleep_time)
exec(_output_dir)